In [1]:
from tifffile import imread, imsave
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import sys
sys.path.append('../')

signal = imread('../data/MRI/signal.tif')

noise = (np.random.randn(2486,256,256)).astype(np.float32)*signal*0.01
noise = gaussian_filter(noise, (0,1,4))*70
noise = noise + np.random.randn(2486,256,256)*signal*0.2
noise = noise-noise.mean()

observation = noise + signal

In [2]:
imsave('../data/MRI/signal_dependent/observation.tif', observation)
imsave('../data/MRI/signal_dependent/noise.tif', observation)

/tmp/ipykernel_1957919/3218207447.py:1: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave('../data/MRI/signal_dependent/observation.tif', observation)
/tmp/ipykernel_1957919/3218207447.py:2: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave('../data/MRI/signal_dependent/noise.tif', observation)


In [1]:
import os
import torch
from tifffile import imread
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping
import sys
sys.path.append('../')
from noise_models.PixelCNN_colearning import PixelCNN
from utils.dataloaders import create_nm_loader
from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

noise = imread('../data/MRI/signal_dependent/noise.tif')

transform = transforms.RandomCrop(64)
train_loader, val_loader, data_mean, data_std = create_nm_loader(noise, batch_size=32, split=0.8, transform=transform)

checkpoint_path = '../nm_checkpoint/MRI/colearning'
trainer = pl.Trainer(default_root_dir=os.path.join(checkpoint_path, "PixelCNN"),
                     gpus=1 if str(device).startswith("cuda") else 0,
                     max_epochs=10,
                     callbacks=[ModelCheckpoint(save_weights_only=False, mode="min", monitor="val_nll"),
                                LearningRateMonitor("epoch"),
                                EarlyStopping('val_nll', patience=10)])

model = PixelCNN(kernel_size = 7,
                 depth = 5, num_filters = 64,
                 num_gaussians = 10,
                 data_mean=data_mean,
                 data_std=data_std).to(device)

trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name       | Type                       | Params
----------------------------------------------------------
0 | v_inconv   | VerticalStackConvolution   | 3.2 K 
1 | h_inconv   | HorizontalStackConvolution | 512   
2 | gatedconvs | ModuleList                 | 479 K 
3 | outconv    | Conv2d                     | 1.9 K 
----------------------------------------------------------
485 K     Trainable params
0         Non-trainable params
485 K     Total params
1.941     Total estimated model params size (MB)
/rds/bear-apps/2021a/EL8-ice/software/pytorch-lightning/1.5.7-foss-2021a-CUDA-11.3.1/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory ../nm_checkpoint/MRI/colearning/PixelCNN/lightning_logs/version_48563610/checkpoints exists and is not empty.

Validation sanity check: 0it [00:00, ?it/s]

/rds/bear-apps/2021a/EL8-ice/software/pytorch-lightning/1.5.7-foss-2021a-CUDA-11.3.1/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/rds/bear-apps/2021a/EL8-ice/software/pytorch-lightning/1.5.7-foss-2021a-CUDA-11.3.1/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [2]:
trainer.save_checkpoint('../nm_checkpoint/MRI/colearning/final_params.ckpt')

In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
import torch
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import sys
sys.path.append('../')
from HDN.models.lvae_colearning import LadderVAE
from utils.dataloaders import create_dn_loader
from tifffile import imread
from noise_models.PixelCNN_colearning import PixelCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

observation = imread('../data/MRI/signal_dependent/observation.tif')

transform = transforms.RandomCrop(64)
train_loader, val_loader, data_mean, data_std, img_shape = create_dn_loader(observation, batch_size=8, split=0.8, transform=transform)
del(observation)

noiseModel = PixelCNN.load_from_checkpoint('../nm_checkpoint/MRI/colearning/final_params.ckpt').to(device).eval()
for param in noiseModel.parameters():
    param.requires_grad = False
    
lr=3e-4
num_latents = 5
z_dims = [32]*int(num_latents)
free_bits = 0.5

model = LadderVAE(z_dims=z_dims,
                   data_mean=data_mean,
                   data_std=data_std,
                   noiseModel=noiseModel,
                   img_shape=img_shape,
                   img_folder='../dn_checkpoint/MRI/colearning/imgs').to(device)

checkpoint_path = '../dn_checkpoint/MRI/colearning'
trainer = pl.Trainer(default_root_dir=os.path.join(checkpoint_path, "HDN"),
                     gpus=1 if str(device).startswith("cuda") else 0,
                     max_epochs=100,
                     logger=False,
                     gradient_clip_val=0,
                     callbacks=[ModelCheckpoint(save_weights_only=False, mode="min", monitor="val_elbo", every_n_epochs=50),
                                EarlyStopping('val_elbo', patience=100)])

trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name             | Type                 | Params
----------------------------------------------------------
0 | first_bottom_up  | Sequential           | 75.8 K
1 | top_down_layers  | ModuleList           | 3.4 M 
2 | bottom_up_layers | ModuleList           | 2.2 M 
3 | final_top_down   | Sequential           | 412 K 
4 | likelihood       | NoiseModelLikelihood | 485 K 
----------------------------------------------------------
6.2 M     Trainable params
485 K     Non-trainable params
6.7 M     Total params
26.625    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

TypeError: forward() missing 1 required positional argument: 'scale'

In [ ]:
trainer.save_checkpoint('../dn_checkpoint/MRI/colearning/final_params.ckpt')

In [16]:
a = [1,2]
b = tuple(a)
c = torch.normal(torch.zeros([3,b[0],b[1]]), 1)

In [15]:
[3,b[0],b[1]]

[3, 1, 2]